In [ ]:
import sqlite3
import pandas as pd
import numpy as np

Build a profile of providers referring patients to the major hospitals in Nashville. Are certain specialties more likely to refer to a particular hospital over the others?

In [ ]:
query = """
SELECT npi, organization_name,  
FROM provider
LEFT JOIN taxonomy
USING(taxonomy_code)
WHERE entity_type_code = 2
AND mailing_address_city_name = 'NASHVILLE'
AND classification = 'General Acute Care Hospital'
"""

with sqlite3.connect('../data/hop_team_db.sqlite') as db: 
    nash_hosp = pd.read_sql(query, db)
    
db.close()

In [ ]:
nash_hosp['organization_name'].value_counts().tail(42)

In [ ]:
query = """
WITH nash_hosp AS (
    SELECT npi AS hosp_npi, organization_name AS hospital
    FROM provider
    LEFT JOIN taxonomy
    USING(taxonomy_code)
    WHERE entity_type_code = 2
    AND mailing_address_city_name = 'NASHVILLE'
    AND classification = 'General Acute Care Hospital'
)
SELECT from_npi, display_name, transaction_count, hospital
FROM referrals
INNER JOIN provider
ON from_npi = npi
INNER JOIN nash_hosp
ON to_npi = hosp_npi
LEFT JOIN taxonomy
USING(taxonomy_code)
WHERE transaction_count >= 50
AND average_day_wait > 50
"""

with sqlite3.connect('../data/hop_team_db.sqlite') as db: 
    nash_hosp_referers = pd.read_sql(query, db)
    
db.close()

In [ ]:
(
    nash_hosp_referers
    .groupby(['display_name', 'hospital'])['transaction_count']
    .sum()
    .to_frame()
    .reset_index()
    .sort_values('transaction_count', ascending = False)
)